In [1]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from joblib import dump, load

In [2]:
df = pd.read_csv('./data-test.csv', index_col=False, encoding='utf-8', error_bad_lines=False)
df.head()

b'Skipping line 16470: expected 7 fields, saw 8\nSkipping line 17495: expected 7 fields, saw 8\nSkipping line 17542: expected 7 fields, saw 11\nSkipping line 20709: expected 7 fields, saw 9\nSkipping line 27366: expected 7 fields, saw 8\nSkipping line 29075: expected 7 fields, saw 8\nSkipping line 30636: expected 7 fields, saw 10\nSkipping line 32731: expected 7 fields, saw 8\nSkipping line 35429: expected 7 fields, saw 8\nSkipping line 47785: expected 7 fields, saw 11\nSkipping line 73959: expected 7 fields, saw 9\nSkipping line 74381: expected 7 fields, saw 8\nSkipping line 77779: expected 7 fields, saw 8\n'


,Unnamed: 0,Comment,Sentiment,Sentiment_Mixed,Sentiment_Negative,Sentiment_Neutral,Sentiment_Positive
0,0,"Yo controlaré tu vida, no pongas resistencia. ...",MIXED,0.6923964023590088,0.0562804751098156,0.11458773910999298,0.13673537969589233
1,1,El Big Brother,NEUTRAL,0.0008369073620997369,0.4505760669708252,0.5237205624580383,0.02486647106707096
2,2,"¡Genial documental como siempre, chicos! Un te...",POSITIVE,0.0011725407093763351,0.0001910059218062088,0.019178738817572594,0.9794577360153198
3,3,No seamos inconscientes comprar al vecino de l...,NEUTRAL,0.17682310938835144,0.02448095567524433,0.4111078381538391,0.3875880241394043
4,4,Capitalismo de vigilancia,NEUTRAL,0.002510450081899762,0.13656044006347656,0.8545124530792236,0.006416601128876209


In [3]:
df = df[df['Sentiment'].isin(['POSITIVE', 'NEGATIVE'])]
df.count()

Unnamed: 0            38418
Comment               38418
Sentiment             38418
Sentiment_Mixed       38418
Sentiment_Negative    38418
Sentiment_Neutral     38418
Sentiment_Positive    38418
dtype: int64

In [4]:
import numpy as np
import re

redisConnection = None

def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

def remove_stopswords(comment):
    stop_words = [ "algún", "alguna", "algunas", "alguno", "algunos", "ambos", "ampleamos", "ante", "antes", "aquel", "aquellas", "aquellos", "aqui", "arriba", "atras", "bajo", "bastante", "bien", "cada", "cierta", "ciertas", "cierto", "ciertos", "como", "con", "conseguimos", "conseguir", "consigo", "consigue", "consiguen", "consigues", "cual", "cuando", "dentro", "desde", "donde", "dos", "el", "ellas", "ellos", "empleais", "emplean", "emplear", "empleas", "empleo", "en", "encima", "entonces", "entre", "era", "eramos", "eran", "eras", "eres", "es", "esta", "estaba", "estado", "estais", "estamos", "estan", "estoy", "fin", "fue", "fueron", "fui", "fuimos", "gueno", "ha", "hace", "haceis", "hacemos", "hacen", "hacer", "haces", "hago", "incluso", "intenta", "intentais", "intentamos", "intentan", "intentar", "intentas", "intento", "ir", "la", "largo", "las", "lo", "los", "mientras", "mio", "modo", "muchos", "muy", "nos", "nosotros", "otro", "para", "pero", "podeis", "podemos", "poder", "podria", "podriais", "podriamos", "podrian", "podrias", "por", "por qué", "porque", "primero", "puede", "pueden", "puedo", "quien", "sabe", "sabeis", "sabemos", "saben", "saber", "sabes", "ser", "si", "siendo", "sin", "sobre", "sois", "solamente", "solo", "somos", "soy", "su", "sus", "también", "teneis", "tenemos", "tener", "tengo", "tiempo", "tiene", "tienen", "todo", "trabaja", "trabajais", "trabajamos", "trabajan", "trabajar", "trabajas", "trabajo", "tras", "tuyo", "ultimo", "un", "una", "unas", "uno", "unos", "usa", "usais", "usamos", "usan", "usar", "usas", "uso", "va", "vais", "valor", "vamos", "van", "vaya", "verdad", "verdadera", "verdadero", "vosotras", "vosotros", "voy", "yo", "él", "ésta", "éstas", "éste", "éstos", "última", "últimas", "último", "últimos", "a", "añadió", "aún", "actualmente", "adelante", "además", "afirmó", "agregó", "ahí", "ahora", "al", "algo", "alrededor", "anterior", "apenas", "aproximadamente", "aquí", "así", "aseguró", "aunque", "ayer", "buen", "buena", "buenas", "bueno", "buenos", "cómo", "casi", "cerca", "cinco", "comentó", "conocer", "consideró", "considera", "contra", "cosas", "creo", "cuales", "cualquier", "cuanto", "cuatro", "cuenta", "da", "dado", "dan", "dar", "de", "debe", "deben", "debido", "decir", "dejó", "del", "demás", "después", "dice", "dicen", "dicho", "dieron", "diferente", "diferentes", "dijeron", "dijo", "dio", "durante", "e", "ejemplo", "ella", "ello", "embargo", "encuentra", "esa", "esas", "ese", "eso", "esos", "está", "están", "estaban", "estar", "estará", "estas", "este", "esto", "estos", "estuvo", "ex", "existe", "existen", "explicó", "expresó", "fuera", "gran", "grandes", "había", "habían", "haber", "habrá", "hacerlo", "hacia", "haciendo", "han", "hasta", "hay", "haya", "he", "hecho", "hemos", "hicieron", "hizo", "hoy", "hubo", "igual", "indicó", "informó", "junto", "lado", "le", "les", "llegó", "lleva", "llevar", "luego", "lugar", "más", "manera", "manifestó", "mayor", "me", "mediante", "mejor", "mencionó", "menos", "mi", "misma", "mismas", "mismo", "mismos", "momento", "mucha", "muchas", "mucho", "nada", "nadie", "ni", "ningún", "ninguna", "ningunas", "ninguno", "ningunos", "no", "nosotras", "nuestra", "nuestras", "nuestro", "nuestros", "nueva", "nuevas", "nuevo", "nuevos", "nunca", "o", "ocho", "otra", "otras", "otros", "parece", "parte", "partir", "pasada", "pasado", "pesar", "poca", "pocas", "poco", "pocos", "podrá", "podrán", "podría", "podrían", "poner", "posible", "próximo", "próximos", "primer", "primera", "primeros", "principalmente", "propia", "propias", "propio", "propios", "pudo", "pueda", "pues", "qué", "que", "quedó", "queremos", "quién", "quienes", "quiere", "realizó", "realizado", "realizar", "respecto", "sí", "sólo", "se", "señaló", "sea", "sean", "según", "segunda", "segundo", "seis", "será", "serán", "sería", "sido", "siempre", "siete", "sigue", "siguiente", "sino", "sola", "solas", "solos", "son", "tal", "tampoco", "tan", "tanto", "tenía", "tendrá", "tendrán", "tenga", "tenido", "tercera", "toda", "todas", "todavía", "todos", "total", "trata", "través", "tres", "tuvo", "usted", "varias", "varios", "veces", "ver", "vez", "y", "ya"]
    stop_words = set(stop_words)
    comment = ' '.join([word for word in comment.split() if word not in stop_words])
    comment = comment.replace('.', '')
    comment = comment.replace(',', '')
    comment = comment.replace(':', '')
    comment = comment.replace(';', '')
    return comment

def normalize(comment):
    comment = str(comment).lower().strip()
    #comment = remove_stopswords(comment)
    return deEmojify(comment)
df['Comment'] = df['Comment'].apply(lambda x: normalize(x))

In [5]:
df['Comment'] = df['Comment'].apply(lambda x: str(x).lower())
df['Sentiment'] = df['Sentiment'].apply(lambda x: 1 if x == 'POSITIVE' else 0)
df.head()

,Unnamed: 0,Comment,Sentiment,Sentiment_Mixed,Sentiment_Negative,Sentiment_Neutral,Sentiment_Positive
2,2,"¡genial documental como siempre, chicos! un te...",1,0.0011725407093763351,0.0001910059218062088,0.019178738817572594,0.9794577360153198
6,6,mas produccion de cosas.... los recursos ya es...,0,0.028857523575425148,0.9441418647766113,0.016397174447774887,0.010603433474898338
8,8,ya valió vreg* :v,0,0.00010315587132936344,0.7937020063400269,0.12492398917675018,0.08127087354660034
9,9,"excelente, muchas gracias!",1,0.0001030199637170881,0.00010250887862639502,0.001561997109092772,0.9982325434684753
12,12,cual cambio revolucionario ? que propones para...,0,0.07215190678834915,0.6406003832817078,0.2784317135810852,0.008816003799438477


In [6]:
vectorizer = CountVectorizer(
    analyzer = 'word',
    lowercase = True,
)
features = vectorizer.fit_transform(
    df['Comment']
)
features_nd = features.toarray() # for easy usage


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test  = train_test_split(
        features_nd, 
        df['Sentiment'],
        train_size=0.80, 
        random_state=1234)

In [9]:
from sklearn.linear_model import LogisticRegression

log_model = LogisticRegression(solver='lbfgs',class_weight='balanced', max_iter=10000)
log_model = log_model.fit(X=X_train, y=y_train)
y_pred = log_model.predict(X_test)
df.head()

,Unnamed: 0,Comment,Sentiment,Sentiment_Mixed,Sentiment_Negative,Sentiment_Neutral,Sentiment_Positive
2,2,"¡genial documental como siempre, chicos! un te...",1,0.0011725407093763351,0.0001910059218062088,0.019178738817572594,0.9794577360153198
6,6,mas produccion de cosas.... los recursos ya es...,0,0.028857523575425148,0.9441418647766113,0.016397174447774887,0.010603433474898338
8,8,ya valió vreg* :v,0,0.00010315587132936344,0.7937020063400269,0.12492398917675018,0.08127087354660034
9,9,"excelente, muchas gracias!",1,0.0001030199637170881,0.00010250887862639502,0.001561997109092772,0.9982325434684753
12,12,cual cambio revolucionario ? que propones para...,0,0.07215190678834915,0.6406003832817078,0.2784317135810852,0.008816003799438477


In [12]:
dump(log_model, 'model2.joblib')
dump(vectorizer, 'vectorizer2.joblib')

['vectorizer2.joblib']

In [13]:
import random
j = random.randint(0,len(X_test)-7)
comments = list(df['Comment'])
for i in range(j,j+7):
    print(y_pred[i])
    ind = features_nd.tolist().index(X_test[i].tolist())
    print(comments[i])

0
algo que jamás en mi perra vida voy a ver en vida real siquiera jaja maldito país tercermundista
0
para q mierda queres tanto si con una 1660 te va igual
1
ese no es el pc más potente para jugar ese i9 9900k no es el más potente ni el mejor :v hay está el i9 9920x
1
un amigo de mi padre tiene una pc con esas estadisticas:
- x2 rtx 2080 de asus
- 32 gb de ram ddr5 corsair
-refrigeracion liquida thermaltake
-i9 9900k
es una hermosura y se puede jugar battlefield 5 en 4k con todo al palo, me parece muy excesivo pero hay gente que le gusta y bueno ahi ellos con su gasto.
saludos desde argentina
0
liiinda las 32 gb ddr5
0
el buen set up de dantegtx
0
yo si veo eso en mi escritorio me pregunto si es un pc o me voy a meter ahi,me van a encoger y mandarme a a la luna xd


In [10]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.8695991671004685
